In [1]:
################ PARAMETERS ################

import numpy as np
import matplotlib.pyplot as plt
import logging
from scipy.interpolate import RegularGridInterpolator
from scipy.special import roots_hermite
from scipy.linalg import eig

def setup():
    
    class par: pass
    
    # Setup specifications in class.
    par.T = 40
    par.rho = 0.75
    par.beta = 0.95
    par.alpha = 1.5
    par.lw = 1.8

    par.h = np.array([0, 0.5, 1])
    par.Nh = len(par.h)
    par.delta = 0.1
    par.phi_1 = 0.2
    par.phi_2 = 0.6

    # 4. taste shocks
    par.sigma_eta = 0.25  # taste shocks

    # 5. income
    par.tax_rate = 0.0
    par.kappa = 1
    par.sigma_xi = 0.1  # capital uncertainty
    par.UB = 0.0 # This might need to be adjusted

    # 6. saving
    par.R = 1.05

    # 7. grids and numerical integration
    par.m_max = 20.0
    par.m_phi = 1.1  # curvature parameters
    par.m_points_low = 50
    par.a_max = 20.0
    par.a_phi = 1.1  # curvature parameters
    par.k_max = 10.0
    par.k_phi = 1.0  # curvature parameters

    # number of elements
    par.Nxi = 8
    par.Nm = 200
    par.Na = 200
    par.Nk = 200
    
    return par

def create_grids(par):

    # 2. Shocks
    par.xi, par.xi_w = GaussHermite_lognorm(par.sigma_xi, par.Nxi)

    # 3. End-of-period assets
    par.grid_a = [nonlinspace(1e-6, par.a_max, par.Na, par.a_phi) for _ in range(par.T)]

    # 4. Cash-on-hand
    par.grid_m = nonlinspace(1e-4, par.m_max, par.Nm, par.m_phi)

    # 5. Human capital
    par.grid_k = nonlinspace(1e-4, par.k_max, par.Nk, par.k_phi)

    return par

In [2]:
################ VARIOUS FUNCTIONS ################

def GaussHermite(n):
    # a. calculations
    i = np.arange(1, n)
    a = np.sqrt(i / 2)
    CM = np.diag(a, 1) + np.diag(a, -1)
    L, V = np.linalg.eig(CM)
    ind = np.argsort(L)
    V = V[:, ind].T
       
    # b. nodes and weights 
    x = L[ind]
    w = np.sqrt(np.pi) * V[:, 0] ** 2
    return x, w

def GaussHermite_lognorm(sigma, n):
    x, w = GaussHermite(n)
    
    x = np.exp(x * np.sqrt(2) * sigma - 0.5 * sigma ** 2)  # mu = -0.5 * sigma^2
    w = w / np.sqrt(np.pi)
    
    # assert a mean of one
    assert abs(1 - np.sum(w * x)) < 1e-8, "Mean of the distribution is not close to 1"
    
    return x, w

def nonlinspace(lo, hi, n, phi):
    assert hi > lo
    assert n >= 2
    assert phi >= 1
    
    # 1. recursion
    x = np.empty(n)
    
    x[0] = lo
    for i in range(1, n):
        x[i] = x[i - 1] + (hi - x[i - 1]) / ((n - i + 1) ** phi)
    
    # 3. assert increaing
    assert np.all(np.diff(x) > 0)
    
    return x

def logsum(v1, v2, sigma):
    """
    Calculates the log-sum and choice-probabilities.
    
    Args:
    v1, v2: column-vectors with the same size.
    sigma: a scalar.

    Returns:
    log_sum: a column vector with the log-sum.
    prob: a matrix with the probabilities.
    """
    
    # 1. setup
    V = np.column_stack((v1, v2))  # stack them next to each other
    
    # 2. maximum over the discrete choices
    mxm = V.max(0)  # mxm is a column vector with the max of each row.
    
    # 3. logsum and probabilities
    if abs(sigma) > 1e-10:
        # numerically robust log-sum
        log_sum = mxm + sigma*(np.log(np.sum(np.exp((V - mxm) / sigma),axis=0)))
    
        # d. numerically robust probability
        prob = np.exp((V- log_sum) / sigma)  
        
    else:  # no smoothing -> max-operator
        id = V.argmax(0)    #Index of maximum
        log_sum = mxm
        prob = np.zeros((v1.size*2))
        I = np.cumsum(np.ones((v1.size,1)))+id*(v1.size)-1
        I = I.astype(int)  # change type to integer
        prob[I] = 1

        prob = np.reshape(prob,(2,v1.size),'A')
    
    return log_sum, prob

def logsum_2(v1, v2, v3, sigma):
    """
    Calculates the log-sum and choice-probabilities.
    
    Args:
    v1, v2, v3: column-vectors with the same size.
    """
    
    # 1. setup
    V = np.column_stack((v1, v2, v3))  # stack them next to each other
    
    # 2. maximum over the discrete choices
    mxm = V.max(0)  # mxm is a column vector with the max of each column.
    
    # 3. logsum and probabilities
    if abs(sigma) > 1e-10:
        # numerically robust log-sum
        log_sum = mxm + sigma * np.log(np.sum(np.exp((V - mxm) / sigma), axis=0))
    
        # numerically robust probability
        prob = np.exp((V - log_sum) / sigma)  
    
    else:  # no smoothing -> max-operator
        id = V.argmax(0)  # Index of maximum
        log_sum = mxm
        prob = np.zeros((v1.size * 3))
        I = np.cumsum(np.ones((v1.size, 1))) + id * v1.size - 1
        I = I.astype(int)  # change type to integer
        prob[I] = 1

        prob = np.reshape(prob, (3, v1.size), 'A')
    
    return log_sum, prob

def logsum_vec(V, sigma):
    """
    Calculates the log-sum and choice-probabilities for a matrix V.

    Args:
    V: a matrix with column vectors stacked next to each other.
    sigma: parameter for the log-sum

    Returns:
    log_sum: log-sum of the values
    prob: probabilities
    """
    
    # 1. setup
    rows, cols = V.shape
    
    # 2. maximum over the discrete choices
    mxm = np.max(V, axis=1, keepdims=True)  # mxm is a column vector with the max of each row.
    
    # 3. logsum and probabilities
    if abs(sigma) > 1.0e-10:
        # a. numerically robust log-sum
        log_sum = mxm + sigma * np.log(np.sum(np.exp((V - mxm) / sigma), axis=1, keepdims=True))
        
        # b. numerically robust probability
        prob = np.exp((V - mxm) / sigma) / np.sum(np.exp((V - mxm) / sigma), axis=1, keepdims=True)

    else:  # no smoothing -> max-operator
        id = np.argmax(V, axis=1)
        log_sum = mxm.flatten()  # pick the max
        prob = np.zeros((rows, cols))  # prob is 1 in one of the cols
        I = np.arange(rows) + (id * rows)  # calculate linear index
        prob.flat[I] = 1
    
    # if we have -inf in the matrix V
    log_sum[np.isnan(log_sum)] = -np.inf
    prob[np.isnan(prob)] = 1 / cols
    
    return log_sum, prob

In [3]:
################ MODELS ################

# Utility functions
def utility(c, h_hour, par):
    return c**(1 - par.rho) / (1 - par.rho) - par.lw * h_hour**par.alpha / par.alpha

def marg_utility_c(c, par):
    return c**(-par.rho)

def inv_marg_utility_c(u, par):
    return u**(-1 / par.rho)

# Transitions
def m_trans(a, k, k_plus, h_choice, par):
    h_hour = par.h[h_choice]
    k = np.tile(k, (1, k_plus.shape[1]))  # Ensure k is broadcast correctly
    m_plus = par.R * a[:, np.newaxis] + par.kappa * (1 - par.tax_rate) * h_hour * k + (h_hour == 0) * par.UB
    return m_plus

def k_trans(k, h_choice, xi, par):
    h_hour = par.h[h_choice]
    k_plus = ((1 - par.delta) * k + par.phi_1 * h_hour**par.phi_2) * xi
    return k_plus

# Expectations
def taste_exp(m_plus, k_plus, v_plus_interp, par):
    v_matrix = np.zeros((m_plus.size, par.Nh))
    for i_nh in range(par.Nh):
        v_interp_values = v_plus_interp[i_nh](np.column_stack((m_plus.ravel(), k_plus.ravel())))
        v_matrix[:, i_nh] = v_interp_values

    V_plus, prob = logsum_vec(v_matrix, par.sigma_eta)
    V_plus = V_plus.reshape(m_plus.shape)
    return V_plus, prob

def future_marg_u(c_plus_interp, m_plus, k_plus, prob, w, par):
    c_plus = []
    for i_nh in range(par.Nh):
        c_plus_i_nh = c_plus_interp[i_nh](np.column_stack((m_plus.ravel(), k_plus.ravel())))
        c_plus.append(c_plus_i_nh.reshape(m_plus.shape))

    marg_u_plus_matrix = np.zeros((m_plus.size, par.Nh))
    for i_nh in range(par.Nh):
        marg_u_plus_matrix[:, i_nh] = marg_utility_c(c_plus[i_nh].ravel(), par)

    marg_u_plus_taste = np.sum(prob * marg_u_plus_matrix, axis=1)
    marg_u_plus_taste = marg_u_plus_taste.reshape(m_plus.shape)

    avg_marg_u_plus = np.sum(w[:, None] * marg_u_plus_taste, axis=0)

    return avg_marg_u_plus

def value_of_choice_gridsearch(C, h, mt, kt, last, par):
    """
    Computes the value of choice given parameters.

    Args:
    C: consumption
    h: hours
    mt: current m values
    kt: current k values
    last: boolean indicating if it is the last period
    par: parameters object

    Returns:
    V: value of the choice
    """
    if last:
        V = utility(C, h, par)
    else:
        K_plus = ((1 - par.delta) * kt + par.phi_1 * par.h[h]**par.phi_2) * par.xi
        kt = np.tile(kt[:, None], (1, K_plus.shape[0]))
        M_plus = par.R * (mt - C) + par.kappa * par.h[h] * kt + (par.h[h] == 0) * par.UB

        V1 = np.zeros(M_plus.size)
        V2 = np.zeros(M_plus.size)
        V3 = np.zeros(M_plus.size)
        
        V1[:] = par.v_plus_interp[0](np.column_stack((M_plus.ravel(), K_plus.ravel())))
        V2[:] = par.v_plus_interp[1](np.column_stack((M_plus.ravel(), K_plus.ravel())))
        V3[:] = par.v_plus_interp[2](np.column_stack((M_plus.ravel(), K_plus.ravel())))
        
        V = utility(C, h, par) + np.sum(par.xi_w * par.beta * logsum_2(V1, V2, V3, par.sigma_eta))
    
    return V

def egm(t, h_choice, k, v_plus_interp, c_plus_interp, par):
    # 1. assets, human capital
    a = par.grid_a[t]
    k = np.full((par.Na, 1), k)
    w = np.tile(par.xi_w, (par.Na, 1))  # Ensure w is broadcast correctly
    xi = np.tile(par.xi, (par.Na, 1))

    # 2. next-period resources and value
    k_plus = k_trans(k, h_choice, xi, par)
    m_plus = m_trans(a, k, k_plus, h_choice, par)

    # Clip m_plus and k_plus
    m_plus = np.clip(m_plus, par.grid_m[0], par.grid_m[-1])
    k_plus = np.clip(k_plus, par.grid_k[0], par.grid_k[-1])

    # Ensure correct broadcasting
    v_plus_vec_raw, prob = taste_exp(m_plus, k_plus, v_plus_interp, par)
    w = w.reshape(v_plus_vec_raw.shape)  # Adjust this based on the logic
    v_plus_raw = np.sum(w * v_plus_vec_raw, axis=1)  # Sum along the correct axis

    # 3. Expected future marginal utility
    avg_marg_u_plus = future_marg_u(c_plus_interp, m_plus, k_plus, prob, w, par)

    # 4. raw c, m, and v
    c_raw = inv_marg_utility_c(par.beta * par.R * avg_marg_u_plus, par)
    m_raw = a[:, np.newaxis] + c_raw  # Adjust the shape of `a` to match `c_raw`

    return c_raw, m_raw, v_plus_raw

def upper_envelope_cpp(c_raw, m_raw, v_plus_raw, h_choice, t, par):

    # 1. Add point at bottom
    c_raw = np.insert(c_raw, 0, 1e-6)
    m_raw = np.insert(m_raw, 0, 1e-6)
    a_raw = np.insert(par.grid_a[t], 0, 0)
    v_plus_raw = np.insert(v_plus_raw, 0, v_plus_raw[0])

    # 2. Call the function
    h_hour = par.h[h_choice]
    c, v = upper_envelope(c_raw, m_raw, v_plus_raw, a_raw, par.grid_m, h_hour, par)

    return c, v

def upper_envelope(c_raw, m_raw, v_plus_raw, a_raw, grid_m, h_hour, par):
    c_raw = np.insert(c_raw, 0, 1e-6)
    m_raw = np.insert(m_raw, 0, 1e-6)
    a_raw = np.insert(a_raw, 0, 0)
    v_plus_raw = np.insert(v_plus_raw, 0, v_plus_raw[0])

    c = np.full(len(grid_m), np.nan)
    v = np.full(len(grid_m), -np.inf)
    
    for i in range(len(a_raw) - 1):
        m_low = m_raw[i]
        m_high = m_raw[i + 1]
        if m_high == m_low:
            continue
        c_slope = (c_raw[i + 1] - c_raw[i]) / (m_high - m_low)

        a_low = a_raw[i]
        a_high = a_raw[i + 1]
        if a_high == a_low:
            continue
        v_plus_slope = (v_plus_raw[i + 1] - v_plus_raw[i]) / (a_high - a_low)

        for j in range(len(grid_m)):
            m_now = grid_m[j]
            do_interp = m_now >= m_low and m_now <= m_high
            do_extrap = m_now > m_high and i == len(a_raw) - 2

            if do_interp or do_extrap:
                c_guess = c_raw[i] + c_slope * (m_now - m_low)
                a_guess = m_now - c_guess
                v_plus = v_plus_raw[i] + v_plus_slope * (a_guess - a_low)
                v_guess = utility(c_guess, h_hour, par) + par.beta * v_plus

                if v_guess > v[j]:
                    v[j] = v_guess
                    c[j] = c_guess

    return c, v


def gridsearch(par, h, last):
    # Initialize V and Cstar
    V = np.zeros((par.Nm, par.Nk))
    Cstar = np.full((par.Nm, par.Nk), np.nan)
    par.grid_C = np.linspace(0, 1, par.Nc)
    
    # Loop over states
    for i_M in range(len(par.grid_m)):
        mt = par.grid_m[i_M]
        
        for i_K in range(len(par.grid_k)):
            kt = par.grid_k[i_K]

            for ic in range(len(par.grid_C)):
                C = par.grid_C[ic] * mt

                # Find value of choice
                V_new = value_of_choice_gridsearch(C, h, mt, kt, last, par)

                # Save if V_new > V
                if V_new > V[i_M, i_K]:
                    V[i_M, i_K] = V_new
                    Cstar[i_M, i_K] = C
    
    return V, Cstar

def solve(par):
    sol = {
        'c': [[None for _ in range(par.Nh)] for _ in range(par.T)],
        'v': [[None for _ in range(par.Nh)] for _ in range(par.T)],
        'v_plus': [[None for _ in range(par.Nh)] for _ in range(par.T)]  # Initialize v_plus with None values
    }

    for i_nh in range(par.Nh):
        sol['c'][par.T - 1][i_nh] = np.tile(par.grid_m, (par.Nk, 1)).T
        sol['v'][par.T - 1][i_nh] = utility(sol['c'][par.T - 1][i_nh], i_nh, par)
        sol['v_plus'][par.T - 1][i_nh] = np.zeros((par.Nm, par.Nk))  # Initialize v_plus with zeros

    c_plus_interp = [None] * par.Nh
    v_plus_interp = [None] * par.Nh
    for t in range(par.T - 2, -1, -1):
        print(t)

        for i_nh in range(par.Nh):
            sol['c'][t][i_nh] = np.full((par.Nm, par.Nk), np.nan)
            sol['v'][t][i_nh] = np.full((par.Nm, par.Nk), np.nan)
            sol['v_plus'][t][i_nh] = np.full((par.Nm, par.Nk), np.nan)  # Initialize v_plus with NaN

        for i_nh in range(par.Nh):
            c_plus_interp[i_nh] = RegularGridInterpolator((par.grid_m, par.grid_k), sol['c'][t + 1][i_nh].T, method='linear')
            v_plus_interp[i_nh] = RegularGridInterpolator((par.grid_m, par.grid_k), sol['v'][t + 1][i_nh].T, method='linear')

        for i_nh in range(par.Nh):
            for i_k in range(par.Nk):
                k = par.grid_k[i_k]

                c_raw, m_raw, v_plus_raw = egm(t, i_nh, k, v_plus_interp, c_plus_interp, par)

                c, v = upper_envelope_cpp(c_raw, m_raw, v_plus_raw, i_nh, t, par)

                sol['c'][t][i_nh][:, i_k] = c
                sol['v'][t][i_nh][:, i_k] = v
                sol['v_plus'][t][i_nh][:, i_k] = v_plus_raw  # Assign v_plus values

    return sol

def sol_dif_pars(par, par_name, par_grid, N, m_ini, k_ini, seed):
    """
    Solves the model for different parameter values and stores the results.

    Args:
    par: parameters object
    par_name: name of the parameter to vary
    par_grid: grid of parameter values
    N: number of simulations
    m_ini: initial m values
    k_ini: initial k values
    seed: random seed

    Returns:
    store: store object with solutions and simulations
    """
    # 1. Initialize
    par.prefix = par_name

    sim = {
        'N': N,
        'm_ini': m_ini,
        'k_ini': k_ini
    }
    
    store = {
        'par_grid': par_grid,
        'par': [],
        'sol': [],
        'sim': []
    }

    # 2. Solve and simulate
    for val in par_grid:
        # Update parameter
        setattr(par, par_name, val)
        store['par'].append(par)

        # Solve
        sol = solve(par)
        store['sol'].append(sol)

        # Simulate
        simulation = simulate(sim, sol, par, seed)
        store['sim'].append(simulation)

    return store

def sol_dif_pars_tax(par, par_name, par_grid, N, m_ini, k_ini, seed, time):
    """
    Solves the model for different parameter values and stores the results.

    Args:
    par: parameters object
    par_name: name of the parameter to vary
    par_grid: grid of parameter values
    N: number of simulations
    m_ini: initial m values
    k_ini: initial k values
    seed: random seed
    time: time parameter for simulate_tax

    Returns:
    store: store object with solutions and simulations
    """
    # 1. Initialize
    par.prefix = f'{par_name}'
    
    sim = {
        'N': N,
        'm_ini': m_ini,
        'k_ini': k_ini
    }
    
    store = {
        'par_grid': par_grid,
        'par': [],
        'sol': [],
        'sim': [None, None]
    }

    # 2. Solve
    for i, val in enumerate(par_grid):
        setattr(par, par_name, val)
        store['par'].append(par)
        store['sol'].append(solve(par))

    # Simulate
    store['sim'][0] = simulate(sim, store['sol'][0], store['par'][0], seed)
    #store['sim'][1] = simulate_tax(sim, store['sol'][0], store['par'][0], store['sol'][1], store['par'][1], seed, time)
    
    return store

def solve_gridsearch(par):
    """
    Solves the model using grid search.

    Args:
    par: parameters object

    Returns:
    sol: solution object
    """
    # 1. C-grid, M-grid and H-grid
    par.grid_C = np.linspace(0, 1, par.Nc)
    
    # 2. Allocate memory
    sol = {
        'c': [[None for _ in range(par.Nh)] for _ in range(par.T)],
        'v': [[None for _ in range(par.Nh)] for _ in range(par.T)]
    }

    par.v_plus_interp = [None] * par.Nh

    # 3. Last period
    for h in range(par.Nh):
        sol['v'][par.T-1][h], sol['c'][par.T - 1][h] = gridsearch(par, h, True)
    
    # 4. Backwards over time
    for t in range(par.T - 2, -1, -1):
        print(t)
        
        # a. Interpolant
        for h in range(par.Nh):
            par.v_plus_interp[h] = RegularGridInterpolator((par.grid_m, par.grid_k), sol['v'][t+1][h], method='linear')

        # b. Find V for all discrete choices and states
        for h in range(par.Nh):
            sol['v'][t][h], sol['c'][t][h] = gridsearch(par, h, False)

    return sol

def euler_error(sim):
    """
    Computes Euler errors for the simulation.

    Args:
    sim: simulation object

    Returns:
    sim: simulation object with added Euler errors
    """
    abs_error = np.abs(sim['lhs_euler'] - sim['rhs_euler'])

    abs_error_0 = abs_error + (abs_error == 0)  # set zeros equal to 1
    log10_abs_error_0 = np.log10(abs_error_0 / sim['c'][:, :-1])  # includes all zeros (log(1) = 0)

    abs_error_nan = np.where(abs_error == 0, np.nan, abs_error)  # set zeros equal to nan
    log10_abs_error_nan = np.log10(abs_error_nan / sim['c'][:, :-1])  # excludes all zeros

    sim['euler_error'] = np.nanmean(np.nanmean(abs_error))
    sim['log10_euler_error'] = np.nanmean(np.nanmean(log10_abs_error_0))
    sim['log10_euler_error_using_nan'] = np.nanmean(np.nanmean(log10_abs_error_nan))
    return sim

def simulate(sim, sol, par, seed):
    """
    Runs the simulation for the given parameters and solutions.

    Args:
    sim: simulation object
    sol: solution object
    par: parameters object
    seed: random seed

    Returns:
    sim: updated simulation object
    """
    # 1. Set seed and initialize
    np.random.seed(seed)
    sim['m'] = sim['m_ini'] * np.ones((sim['N'], par.T))
    sim['k'] = sim['k_ini'] * np.ones((sim['N'], par.T))

    # 2. Allocate
    sim['c'] = np.full((sim['N'], par.T), np.nan)
    sim['h_choice'] = np.full((sim['N'], par.T), np.nan)
    sim['a'] = np.zeros((sim['N'], par.T))
    sim['lhs_euler'] = np.full((sim['N'], par.T-1), np.nan)
    sim['rhs_euler'] = np.full((sim['N'], par.T-1), np.nan)

    # 3. Random draws
    unif = np.random.rand(sim['N'], par.T)
    shock = np.exp(np.random.randn(sim['N'], par.T-1) * par.sigma_xi)

    # 4. Simulate
    for t in range(par.T):
        # a. Values of discrete choices and interpolants
        v_matrix = np.full((sim['m'][:, t].size, par.Nh), np.nan)
        c_interp = [None] * par.Nh
        c_plus_interp = [None] * par.Nh

        for i_nh in range(par.Nh):
            v_interp = RegularGridInterpolator((par.grid_m, par.grid_k), sol['v'][t][i_nh], method='linear')
            v_matrix[:, i_nh] = v_interp((sim['m'][:, t], sim['k'][:, t]))

            c_interp[i_nh] = RegularGridInterpolator((par.grid_m, par.grid_k), sol['c'][t][i_nh], method='linear')
            if t < par.T - 1:
                c_plus_interp[i_nh] = RegularGridInterpolator((par.grid_m, par.grid_k), sol['c'][t+1][i_nh], method='linear')

        # b. Choice-specific probabilities
        _, prob = logsum_vec(v_matrix, par.sigma_eta)

        # c. Actual labour choice
        prob_cum = np.cumsum(prob, axis=1)  # cumsum across cols, so last col equals 1
        I = np.sum(unif[:, t][:, None] > prob_cum, axis=1) + 1  # find the first col of prob_cum that exceeds unif
        sim['h_choice'][:, t] = I  # 1 is no labour, last is full labour

        # Ensure h_choice is an integer array
        h_choice = sim['h_choice'][:, t].astype(int) - 1  # Subtract 1 to match Python's 0-based indexing

        # d. Actual consumption choice
        for i_nh in range(par.Nh):
            ind = (h_choice == i_nh)  # find indices where the agent choice is i_nh
            sim['c'][ind, t] = c_interp[i_nh]((sim['m'][ind, t], sim['k'][ind, t]))

        # e. Next period
        if t < par.T - 1:
            # 1. Update states
            sim['a'][:, t] = sim['m'][:, t] - sim['c'][:, t]

            a = sim['a'][:, t]
            k = sim['k'][:, t].reshape(-1, 1)  # Ensure k has the correct shape for broadcasting
            xi = shock[:, t]

            k_plus = k_trans(k, h_choice, xi, par)
            m_plus = m_trans(a, k, k_plus, h_choice, par)

            sim['k'][:, t+1] = k_plus
            sim['m'][:, t+1] = m_plus

            # 2. Expected future marginal utility
            avg_marg_u_plus = future_marg_u(c_plus_interp, m_plus, k_plus, prob, 1, par)

            # 3. LHS and RHS of Euler equation
            sim['lhs_euler'][:, t] = marg_utility_c(sim['c'][:, t], par)
            sim['rhs_euler'][:, t] = par.beta * par.R * avg_marg_u_plus

            # 4. Ignore corner solutions
            corner_ind = (a < 1e-5)
            sim['lhs_euler'][corner_ind, t] = np.nan
            sim['rhs_euler'][corner_ind, t] = np.nan

    # 5. Calculate moments/statistics
    # i. Labor supply
    sim['labor'] = np.full((par.Nh, par.T), np.nan)
    for i in range(par.Nh):
        sim['labor'][i, :] = np.sum(sim['h_choice'] == i + 1, axis=0) / sim['N']

    sim['means'] = {
        'hours': np.sum(par.h * sim['labor'], axis=0),
        'cons': np.mean(sim['c'], axis=0),
        'assets': np.mean(sim['a'], axis=0),
        'cash': np.mean(sim['m'], axis=0),
        'capital': np.mean(sim['k'], axis=0),
        'wage': np.mean(par.kappa * sim['k'], axis=0)
    }

    # vii. Euler errors
    sim = euler_error(sim)

    return sim


In [4]:
################ FIGURES ################

def color():
    return ['k', 'b', 'r', 'g', 'y', 'c', 'm', (102/255, 0/255, 51/255), (153/255, 1, 0)]

def sim_choice_fig(par, sim):
    fig = plt.figure()
    fig.canvas.manager.set_window_title(f"labor_choice_{par['prefix']}")
    colors = color()
    
    for i in range(par['Nh']):
        h = par['h'][i]
        plt.plot(range(1, par['T']+1), sim['labor'][i, :], '-o',
                 linewidth=1.5, markersize=3, color=colors[i],
                 label=f'$h_t = {h:.2f}$')
    
    plt.xlabel('$t$')
    plt.legend(loc='best')
    plt.box('on')
    plt.grid(True)
    
    printfig(fig)

def sim_mean_fig_new(var, par):
    fig = plt.figure()
    fig.canvas.manager.set_window_title(f"State_variable_{par['prefix']}")
    colors = color()
    
    plt.plot(np.mean(var, axis=0), '-o', linewidth=1.5, markersize=3, color=colors[0], label='Mean')
    plt.plot(np.percentile(var, 2.5, axis=0), '-o', linewidth=1.5, markersize=3, color=colors[1], label='2.5 percentile')
    plt.plot(np.percentile(var, 97.5, axis=0), '-o', linewidth=1.5, markersize=3, color=colors[2], label='97.5 percentile')
    
    plt.xlabel('$t$')
    plt.legend(loc='best')
    plt.grid(True)
    
    printfig(fig)

def sim_mean_fig(par, sim, vars):
    colors = color()
    
    for var_group in vars:
        fig = plt.figure()
        fig.canvas.manager.set_window_title(f"mean_{''.join(var_group)}_{par['prefix']}")
        for j, var in enumerate(var_group):
            plt.plot(range(1, par['T']+1), sim['means'][var], '-o',
                     linewidth=1.5, markersize=3, color=colors[j],
                     label=f'${var}$')
        
        plt.xlabel('$t$')
        plt.legend(loc='best')
        plt.box('on')
        plt.grid(True)
        
        printfig(fig)

def sim_mean_dif_pars(store, vars):
    colors = color()
    
    for i, var in enumerate(vars):
        fig = plt.figure()
        fig.canvas.manager.set_window_title(f"mean_{var}_{store['par'][0]['prefix']}")
        for j, par_grid_val in enumerate(store['par_grid']):
            if len(par_grid_val) > 1:
                par_grid_val = len(par_grid_val)
            
            plt.plot(range(1, store['par'][0]['T']+1), store['sim'][j]['means'][var], '-o',
                     linewidth=1.5, markersize=3, color=colors[j],
                     label=f'${store["par"][0]["prefix"]}={par_grid_val:.2f}$')
        
        plt.xlabel('$t$')
        
        if i == 0:
            plt.ylabel('$Mean(h_t)$')
        elif i == 1:
            plt.ylabel('$Mean(K_t)$')
        
        plt.legend(loc='best')
        plt.box('on')
        plt.grid(True)
        
        printfig(fig)

def sim_mean_hhours(sim, par):
    fig = plt.figure()
    fig.canvas.manager.set_window_title(f"mean_lhours_{par['prefix']}")
    
    labor = np.zeros((par['Nh'], par['T']))
    for i in range(par['Nh']):
        labor[i, :] = np.sum(sim['h_choice'] == i, axis=0) / sim['N']
    
    mean_hours = np.sum(par['h'][:, None] * labor, axis=0)
    plt.plot(mean_hours, '-o', color='k', linewidth=1.5, markersize=3)
    plt.ylabel('$Mean(h_t)$')
    plt.xlabel('$t$')
    plt.grid(True)
    
    printfig(fig)

def sim_mean_hhours_vary_D(sim, sim_alternative, D, par):
    fig = plt.figure()
    fig.canvas.manager.set_window_title(f"mean_lhours_{par['prefix']}")
    
    h_three = np.array([0, 0.5, 1])
    labor = np.zeros((3, par['T']))
    for i in range(3):
        labor[i, :] = np.sum(sim['h_choice'] == i, axis=0) / sim['N']
    
    mean_hours = np.sum(h_three[:, None] * labor, axis=0)
    plt.plot(mean_hours, '-o', color='k', label='$D = 3$', linewidth=1.5, markersize=3)
    plt.plot(sim_alternative, '-o', color='r', label=f'$D = {D}$', linewidth=1.5, markersize=3)
    
    plt.ylabel('$Mean(h_t)$')
    plt.xlabel('$t$')
    plt.legend(loc='best')
    plt.grid(True)
    
    printfig(fig)

def printfig(fig):
    fig.tight_layout()
    plt.show()


In [5]:
import numpy as np
import random

# Set random seed
seed = 2017
np.random.seed(seed)
random.seed(seed)

# Main execution
par = setup()
par = create_grids(par)
par.prefix = ''

# Function to solve the model using EGM (this function should be defined elsewhere)
sol = solve(par)

print("Parameters:", vars(par))
print("Solution:", sol)


38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Parameters: {'__module__': '__main__', '__dict__': <attribute '__dict__' of 'par' objects>, '__weakref__': <attribute '__weakref__' of 'par' objects>, '__doc__': None, 'T': 40, 'rho': 0.75, 'beta': 0.95, 'alpha': 1.5, 'lw': 1.8, 'h': array([0. , 0.5, 1. ]), 'Nh': 3, 'delta': 0.1, 'phi_1': 0.2, 'phi_2': 0.6, 'sigma_eta': 0.25, 'tax_rate': 0.0, 'kappa': 1, 'sigma_xi': 0.1, 'UB': 0.0, 'R': 1.05, 'm_max': 20.0, 'm_phi': 1.1, 'm_points_low': 50, 'a_max': 20.0, 'a_phi': 1.1, 'k_max': 10.0, 'k_phi': 1.0, 'Nxi': 8, 'Nm': 200, 'Na': 200, 'Nk': 200, 'xi': array([0.65740519, 0.75182734, 0.84480292, 0.94279352, 1.05012372,
       1.17192993, 1.31685799, 1.50599637]), 'xi_w': array([1.12614538e-04, 9.63522012e-03, 1.17239908e-01, 3.73012258e-01,
       3.73012258e-01, 1.17239908e-01, 9.63522012e-03, 1.12614538e-04]), 'grid_a': [array([1.00000000e-06, 5.88713989e-02, 1.17893049e-01, 1.77067018e

In [6]:
# Simulate
sim = {
    'N': int(1000),  # 10^5
    'm_ini': 1.5,
    'k_ini': 1
}

# Function to simulate the model (this function should be defined elsewhere)
sim = simulate(sim, sol, par, seed)

# Calculate Euler error (this method should be defined as part of the simulation)
euler = sim['log10_euler_error_using_nan']

# Print outputs for verification (Optional)
print("Simulation Results:", sim)
print("Euler Error:", euler)


ValueError: could not broadcast input array from shape (1000,1000) into shape (1000,)